In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
import re
import jieba

In [2]:
# 数据整合，输出新数据
def merge_file(file1,file2):
    feature = pd.read_csv(file1,sep=",")
    label = pd.read_csv(file2,sep=",")
    data = feature.merge(label,on='id') # 按照id列整合两个训练集表
    data["X"] = data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1) # "title","content"两列合为一列"X"
    df = data[['id', 'title', 'content', 'label', 'X']]
    return df

data = merge_file("Train_DataSet.csv","Train_DataSet_Label.csv")

In [3]:
order = ['id', 'title', 'content', 'X', 'label']
data = data[order]
data

,id,title,content,X,label
0,7a3dd79f90ee419da87190cff60f7a86,问责领导(上黄镇党委书记张涛，宣国才真能一手遮天吗？),这几天看了有人举报施某某的贴子，经与举报人联系证实，是宣某当天中午请举报人和枪手喝酒后，晚上...,问责领导(上黄镇党委书记张涛，宣国才真能一手遮天吗？)这几天看了有人举报施某某的贴子，经与举...,2
1,7640a5589bc7486ca199eeeb38af79dd,江歌事件:教会孩子，善良的同时更要懂得保护自己!,过去一年的江歌悲剧，这几日再次刷屏：住在东京都中野区的中国女留学生江歌，收留了被前男友陈世锋...,江歌事件:教会孩子，善良的同时更要懂得保护自己!过去一年的江歌悲剧，这几日再次刷屏：住在东京...,1
2,8c5bda93e4ba401f90a0faa5b28fe57f,"绝味鸭脖广告""开黄腔""引众怒 ""双11""这么拼值吗?",“双11”1600亿的销售额让中国乃至全世界感到震惊，为此，不仅有不惜欠债百万的“剁手党”，...,"绝味鸭脖广告""开黄腔""引众怒 ""双11""这么拼值吗?“双11”1600亿的销售额让中国乃至全...",2
3,1aa777fed31a4b8a9d866f05b5477557,央视曝光!如东一医药企业将槽罐车改成垃圾车，夜间偷排高浓度废水,2016年以来，如东高新区一家叫做拜瑞生物医药的企业竟然将装废水的槽罐车伪装成洒水车，常年在...,央视曝光!如东一医药企业将槽罐车改成垃圾车，夜间偷排高浓度废水2016年以来，如东高新区一家...,2
4,6c67ac55360340258e157f3710ebae6c,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发了!,新闻资讯•民生热点•城市生活•南通人都在看浏览更多资讯请关注微信公众号:南通传媒网昨晚央视一...,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发...,2
5,4760de6f94e249cd8d8cd5634fd8dc5a,央视曝光!南通一医药企业夜间偷排高浓度废水...丢脸!,昨晚央视一套晚间新闻栏目曝光了一则新闻报道的对象竟是如东一家企业的事情▽2016年以来，南通...,央视曝光!南通一医药企业夜间偷排高浓度废水...丢脸!昨晚央视一套晚间新闻栏目曝光了一则新闻...,2
6,8c35b37b2a8a4884b3b8071b879e6fa6,粉丝爆料:五洲国际无锡项目涉嫌诈骗、非法集资,随着金融街1号（jinrongjieyihao）对有非法集资、诈骗嫌疑的案例的曝光，粉丝们的...,粉丝爆料:五洲国际无锡项目涉嫌诈骗、非法集资随着金融街1号（jinrongjieyihao）...,2
7,3b27c107b5134e9dabda161b4fb2c353,年内约10起锂电重组失败 资本对高估值收购说“不”,摘要来自中国电池联盟的数据显示，今年以来，国内动力电池产业链投资额已接近千亿元，相关兼并购案...,年内约10起锂电重组失败 资本对高估值收购说“不”摘要来自中国电池联盟的数据显示，今年以来，...,1
8,167c5e0e39164b5fad96391422cc166d,男子梦想一夜暴富持水泥块砸机,一分钱都没取到 近日，江苏扬州，谢某为盗取钱款手持水泥块疯砸机。由于机自带防盗功能，谢某未...,男子梦想一夜暴富持水泥块砸机 一分钱都没取到 近日，江苏扬州，谢某为盗取钱款手持水泥块疯砸机...,2
9,51d757b893294b2693ddd16d94356d2d,北京多家法院供暖纠纷:案件主体为供暖费追缴山海网,北京的这个冬天来得比往年更早一些，大风寒潮一次次刷新着温度的新低。冬季采暖直接关系着居民的...,北京多家法院供暖纠纷:案件主体为供暖费追缴山海网 北京的这个冬天来得比往年更早一些，大风寒潮...,2


In [4]:
data['X'][0]

'问责领导(上黄镇党委书记张涛，宣国才真能一手遮天吗？)这几天看了有人举报施某某的贴子，经与举报人联系证实，是宣某当天中午请举报人和枪手喝酒后，晚上才发的贴子！ 本人不去讨论前二天的举报，相信总归会有说法的！ 今天一看施全军2017年1月2日实名举报上黄镇宣国才的贴子（仍被锁定禁止评论）已经正好一整年了 =750) window.open(\'http://img.jsly001.com/attachment/mon_1801/4_291085_c796a6a86e17121.jpg?123\');" onload="if(this.offsetwidth>\'750\')this.width=\'750\';" src="http://img.jsly001.com/attachment/mon_1801/4_291085_c796a6a86e17121.jpg?123" style="max-width:750px;"/>图片:/home/alidata/www/data/tmp/qfupload/4_291085_1514981471478952.jpg 施全军实名举报50天后，上黄镇党委政府回复如下图： =750) window.open(\'http://img.jsly001.com/attachment/mon_1801/4_291085_a9b11b7ea2b1ce9.jpg?90\');" onload="if(this.offsetwidth>\'750\')this.width=\'750\';" src="http://img.jsly001.com/attachment/mon_1801/4_291085_a9b11b7ea2b1ce9.jpg?90" style="max-width:750px;"/>图片:/home/alidata/www/data/tmp/qfupload/4_291085_1514981472631668.jpg =750) window.open(\'http://img.jsly001.com/attachment/mon_1801/4_291085_9cde9b3943fe20c.jpg?75\');" onload="if(this.offsetwidth>\'750\')this.width=\'750\

In [33]:
Data = data

In [34]:
traindata = Data.sample(frac=0.9, replace=False, random_state=42, axis=0)

In [35]:
traindata

,id,title,content,X,label
997,3842bacd1e614f219ae793708f3d53ad,常州市钟楼区人民法院-保险合同纠纷,常州市钟楼区人民法院-保险合同纠纷\n直播回顾\n保险合同纠纷\n审理法院:常州市钟楼区人民...,常州市钟楼区人民法院-保险合同纠纷常州市钟楼区人民法院-保险合同纠纷\n直播回顾\n保险合同...,1
736,a66f9b6731e44f3d9dc9714368d6a4db,这次我真心傻逼了，被彩票平台使用宫心计骗了36000 - 赚客..._赚客吧,2018，5，16突然接到一个电话说帮忙打彩票一单给200元什么都不要只要帮忙刷流水到156...,这次我真心傻逼了，被彩票平台使用宫心计骗了36000 - 赚客..._赚客吧2018，5，1...,2
4076,c945dbdc148b4c999e715fdec62042f5,“红色天使 热血先锋”被泰安市直机关工委评为优秀服务品牌,大众网泰安5月23日讯（通讯员 马克）5月22日，泰安市中心血站党支部创建的“红色天使 热血...,“红色天使 热血先锋”被泰安市直机关工委评为优秀服务品牌大众网泰安5月23日讯（通讯员 马克...,0
1670,dbb3a2b4012d4a9ca6f66371910cf3e9,蓝色引领 创新发展 ——2017中国海洋经济博览会12月14日举..._搜狐,中国海博会在国家和广东省的高度重视下，湛江市和各承办单位通力合作，推动中国海博会越办越好，逐...,蓝色引领 创新发展 ——2017中国海洋经济博览会12月14日举..._搜狐中国海博会在国家...,1
3308,6acf63d063f542dfa08edc196fbaea4c,遇到糟糕的上级，你该怎么办?,大家好，我是职场一年级的班长，今天是和大家一起学习的第?45?天。职场上我们可以控制自己，但...,遇到糟糕的上级，你该怎么办?大家好，我是职场一年级的班长，今天是和大家一起学习的第?45?天...,1
1575,18afe169f70c4ba6940815ce8d9e0154,来分期套现方法_浙江新闻,首页>>新闻大观>>国内新闻>>南昌市白条套现方法中国有34个省市自治区(包括香港、澳门、台...,来分期套现方法_浙江新闻首页>>新闻大观>>国内新闻>>南昌市白条套现方法中国有34个省市自...,1
230,5a2e33701f8b439e81631b80f7a0c0fb,县检察院:判决免刑不当 抗诉发回重审_ _检察天地_平利政法网,2017年6月，被告人孙某酒后驾驶摩托车与刘某驾驶的摩托车发生碰撞，引发交通事故，经交警队认...,县检察院:判决免刑不当 抗诉发回重审_ _检察天地_平利政法网2017年6月，被告人孙某酒后...,2
4072,b302fe1e6679460083741964310f012d,持续发力整治交通乱象,为进一步整治城区交通乱象，持续保持对交通违法乱象整治的高压，借鉴中卫市交通管理经验，结合辖区...,持续发力整治交通乱象为进一步整治城区交通乱象，持续保持对交通违法乱象整治的高压，借鉴中卫市交...,1
2303,938c8aa432914e988d85dfa8ed9f3e53,这邻国突然要升级真航母，我们真得留点神！_搜狐军事_搜狐网,原标题：这邻国突然要升级真航母，我们真得留点神！出云级直升机驱逐舰是日本现役排水量最大的水面...,这邻国突然要升级真航母，我们真得留点神！_搜狐军事_搜狐网原标题：这邻国突然要升级真航母，我...,1
6129,8c19c998dee944e6aef99570ad535898,12层高楼三秒爆破倒塌 再见！吴中国贸大厦,"20号晚上，始建于1996年的苏州吴中国贸大厦成功爆破。这座安静坚守了22年的大楼至此""尘封...",12层高楼三秒爆破倒塌 再见！吴中国贸大厦20号晚上，始建于1996年的苏州吴中国贸大厦成功...,2


In [36]:
testdata = Data[~Data.index.isin(traindata.index)]
testdata

,id,title,content,X,label
4,6c67ac55360340258e157f3710ebae6c,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发了!,新闻资讯•民生热点•城市生活•南通人都在看浏览更多资讯请关注微信公众号:南通传媒网昨晚央视一...,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发...,2
16,aba9cf6ec2b5421ab944878d6a9d1b23,这伙是他自已说的教育废品，文不能作文。一篇文章写着写着就不知道是跑到了天涯还是奔向了海角。教...,这伙是他自已说的教育废品，文不能作文。一篇文章写着写着就不知道是跑到了天涯还是奔向了海角。...,这伙是他自已说的教育废品，文不能作文。一篇文章写着写着就不知道是跑到了天涯还是奔向了海角。教...,2
34,e51bba6b26bf42e49e8d6702cf20747a,驾驶员操作失误液化气泄漏燃烧爆炸致人死亡搜狐新闻搜狐网,法制网济南月日电记者徐鹏记者今天从山东省安监局获悉，山东省政府于近日批复了临沂金誉石化有限...,驾驶员操作失误液化气泄漏燃烧爆炸致人死亡搜狐新闻搜狐网 法制网济南月日电记者徐鹏记者今天从山...,2
64,ccadc08b93d2400bb73a7f104132bea2,业内人说：恒大学万达，文化产业岂止于亏损？,王健林在地产之外，打造了一个“文化产业帝国”，如今，许家印也要这么做。\n \n \n\t今...,业内人说：恒大学万达，文化产业岂止于亏损？王健林在地产之外，打造了一个“文化产业帝国”，如今...,1
98,601ace4e5c0744268dedc89d66a765bb,南京一高级中学爆发水痘 疫情班级已被隔离_新浪江苏_新浪网,最近有家长向我们反映，南京大厂高级中学大面积爆发水痘。事实情况究竟如何，今天(11月14日)...,南京一高级中学爆发水痘 疫情班级已被隔离_新浪江苏_新浪网最近有家长向我们反映，南京大厂高级...,2
114,9de6644e2bf448e6b381b64fd1b1b782,手机突然没信号？小心了！有人因此损失24万,遇到手机突然没信号，或是无故没服务需警惕。武汉一女子出差遇到类似情形，两天后发现被冒名补卡转...,手机突然没信号？小心了！有人因此损失24万遇到手机突然没信号，或是无故没服务需警惕。武汉一女...,2
116,e1f60a61b72749ffafbce2266100ffb3,安康一九零后青年盗窃摩托车 千里走单骑,内容摘要： 2017年8月29日，家住陕西省白河县西营镇村民嫌疑人张某（1992年出生）在镇...,安康一九零后青年盗窃摩托车 千里走单骑内容摘要： 2017年8月29日，家住陕西省白河县西营...,2
119,cef1971997da4350bd4448bccf468dd3,杨光富、李锦蓬走私、贩卖、运输、制造毒品二审刑事判决书 福建省...,上诉人(原审被告人)杨光富，男，1967年3月18日出生于福建省浦城县，汉族，初中文化，无业...,杨光富、李锦蓬走私、贩卖、运输、制造毒品二审刑事判决书 福建省...上诉人(原审被告人)杨光...,2
127,88fbb71a8e58470db7d37f4edf514299,[百姓心声]中国移动的信号越来越差，垃圾移动，携号转网一出来立马投奔电信,不知道在钦州城区的朋友有没有发现，中国移动的信号越来越垃圾，本人手机是g9008v的，可以同...,[百姓心声]中国移动的信号越来越差，垃圾移动，携号转网一出来立马投奔电信不知道在钦州城区的朋...,2
130,717d40edb18b404a94c3fb0856ccfdd2,网曝盖网商城涉嫌传销曝光 李绍增、郑永雄原始股骗局贾格丽-浦口...,到7月下旬，就有人问，不管是国内上市，盖网在广州开了万人大会。其非法发行股票，奖励也会相应提...,网曝盖网商城涉嫌传销曝光 李绍增、郑永雄原始股骗局贾格丽-浦口...到7月下旬，就有人问，不...,2


In [40]:
traindata.to_csv("train_train2.csv",index=False, encoding='utf-8')
testdata.to_csv("train_dev2.csv",index=False, encoding='utf-8')


In [41]:
testdata

,id,title,content,X,label
4,6c67ac55360340258e157f3710ebae6c,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发了!,新闻资讯•民生热点•城市生活•南通人都在看浏览更多资讯请关注微信公众号:南通传媒网昨晚央视一...,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发...,2
16,aba9cf6ec2b5421ab944878d6a9d1b23,这伙是他自已说的教育废品，文不能作文。一篇文章写着写着就不知道是跑到了天涯还是奔向了海角。教...,这伙是他自已说的教育废品，文不能作文。一篇文章写着写着就不知道是跑到了天涯还是奔向了海角。...,这伙是他自已说的教育废品，文不能作文。一篇文章写着写着就不知道是跑到了天涯还是奔向了海角。教...,2
34,e51bba6b26bf42e49e8d6702cf20747a,驾驶员操作失误液化气泄漏燃烧爆炸致人死亡搜狐新闻搜狐网,法制网济南月日电记者徐鹏记者今天从山东省安监局获悉，山东省政府于近日批复了临沂金誉石化有限...,驾驶员操作失误液化气泄漏燃烧爆炸致人死亡搜狐新闻搜狐网 法制网济南月日电记者徐鹏记者今天从山...,2
64,ccadc08b93d2400bb73a7f104132bea2,业内人说：恒大学万达，文化产业岂止于亏损？,王健林在地产之外，打造了一个“文化产业帝国”，如今，许家印也要这么做。\n \n \n\t今...,业内人说：恒大学万达，文化产业岂止于亏损？王健林在地产之外，打造了一个“文化产业帝国”，如今...,1
98,601ace4e5c0744268dedc89d66a765bb,南京一高级中学爆发水痘 疫情班级已被隔离_新浪江苏_新浪网,最近有家长向我们反映，南京大厂高级中学大面积爆发水痘。事实情况究竟如何，今天(11月14日)...,南京一高级中学爆发水痘 疫情班级已被隔离_新浪江苏_新浪网最近有家长向我们反映，南京大厂高级...,2
114,9de6644e2bf448e6b381b64fd1b1b782,手机突然没信号？小心了！有人因此损失24万,遇到手机突然没信号，或是无故没服务需警惕。武汉一女子出差遇到类似情形，两天后发现被冒名补卡转...,手机突然没信号？小心了！有人因此损失24万遇到手机突然没信号，或是无故没服务需警惕。武汉一女...,2
116,e1f60a61b72749ffafbce2266100ffb3,安康一九零后青年盗窃摩托车 千里走单骑,内容摘要： 2017年8月29日，家住陕西省白河县西营镇村民嫌疑人张某（1992年出生）在镇...,安康一九零后青年盗窃摩托车 千里走单骑内容摘要： 2017年8月29日，家住陕西省白河县西营...,2
119,cef1971997da4350bd4448bccf468dd3,杨光富、李锦蓬走私、贩卖、运输、制造毒品二审刑事判决书 福建省...,上诉人(原审被告人)杨光富，男，1967年3月18日出生于福建省浦城县，汉族，初中文化，无业...,杨光富、李锦蓬走私、贩卖、运输、制造毒品二审刑事判决书 福建省...上诉人(原审被告人)杨光...,2
127,88fbb71a8e58470db7d37f4edf514299,[百姓心声]中国移动的信号越来越差，垃圾移动，携号转网一出来立马投奔电信,不知道在钦州城区的朋友有没有发现，中国移动的信号越来越垃圾，本人手机是g9008v的，可以同...,[百姓心声]中国移动的信号越来越差，垃圾移动，携号转网一出来立马投奔电信不知道在钦州城区的朋...,2
130,717d40edb18b404a94c3fb0856ccfdd2,网曝盖网商城涉嫌传销曝光 李绍增、郑永雄原始股骗局贾格丽-浦口...,到7月下旬，就有人问，不管是国内上市，盖网在广州开了万人大会。其非法发行股票，奖励也会相应提...,网曝盖网商城涉嫌传销曝光 李绍增、郑永雄原始股骗局贾格丽-浦口...到7月下旬，就有人问，不...,2
